In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from sqlalchemy import create_engine
! pip install psycopg2-binary

# Extract CSVs into DataFrames

In [2]:
#Path to Salary CSV file 
salary_path = "Data/salary_potential.csv"

#Read the CSV file and create dataframe
salary_data = pd.read_csv(salary_path)

salary_data.head()

,rank,name,state_name,early_career_pay,mid_career_pay,make_world_better_percent,stem_percent
0,1,Auburn University,Alabama,54400,104500,51.0,31
1,2,University of Alabama in Huntsville,Alabama,57500,103900,59.0,45
2,3,The University of Alabama,Alabama,52300,97400,50.0,15
3,4,Tuskegee University,Alabama,54500,93500,61.0,30
4,5,Samford University,Alabama,48400,90500,52.0,3


In [3]:
#Path to Tuition CSV file 
tuition_path = "Data/tuition_cost.csv"

#Read the CSV file and create dataframe
tuition_data = pd.read_csv(tuition_path)

tuition_data.head()

,name,state,state_code,type,degree_length,room_and_board,in_state_tuition,in_state_total,out_of_state_tuition,out_of_state_total
0,Aaniiih Nakoda College,Montana,MT,Public,2 Year,NaN,2380,2380,2380,2380
1,Abilene Christian University,Texas,TX,Private,4 Year,10350.0,34850,45200,34850,45200
2,Abraham Baldwin Agricultural College,Georgia,GA,Public,2 Year,8474.0,4128,12602,12550,21024
3,Academy College,Minnesota,MN,For Profit,2 Year,NaN,17661,17661,17661,17661
4,Academy of Art University,California,CA,For Profit,4 Year,16648.0,27810,44458,27810,44458


# Transform DataFrames

In [4]:
#Clean up the dataframe
salary= salary_data.drop(columns= ["rank","stem_percent","make_world_better_percent"])
salary.head()

,name,state_name,early_career_pay,mid_career_pay
0,Auburn University,Alabama,54400,104500
1,University of Alabama in Huntsville,Alabama,57500,103900
2,The University of Alabama,Alabama,52300,97400
3,Tuskegee University,Alabama,54500,93500
4,Samford University,Alabama,48400,90500


In [5]:
#Clean up the dataframe
tuition_data_clean = tuition_data.drop(columns= ["in_state_total","out_of_state_tuition","state_code","room_and_board"])

tuition_frame= tuition_data_clean.dropna()

tuition = tuition_frame[tuition_frame['name'].isin(salary['name'])]
tuition.head()

,name,state,type,degree_length,in_state_tuition,out_of_state_total
5,Adams State University,Colorado,Public,4 Year,9440,29238
10,Adventist University of Health Sciences,Florida,Private,4 Year,15150,19350
11,Agnes Scott College,Georgia,Private,4 Year,41160,53490
16,Alabama State University,Alabama,Public,4 Year,11068,24818
19,Alaska Pacific University,Alaska,Private,4 Year,20830,28130


In [6]:
#Merge the salary and tuition df
sal_tuition = pd.merge(tuition, salary, on='name')
sal_tuition.drop(columns=["state"])

,name,type,degree_length,in_state_tuition,out_of_state_total,state_name,early_career_pay,mid_career_pay
0,Adams State University,Public,4 Year,9440,29238,Colorado,44400,81400
1,Adventist University of Health Sciences,Private,4 Year,15150,19350,Florida,51600,89800
2,Agnes Scott College,Private,4 Year,41160,53490,Georgia,46000,83600
3,Alabama State University,Public,4 Year,11068,24818,Alabama,39800,71500
4,Alaska Pacific University,Private,4 Year,20830,28130,Alaska,50300,90000
...,...,...,...,...,...,...,...,...
723,Worcester Polytechnic Institute,Private,4 Year,50530,65304,Massachusetts,73600,135500
724,Xavier University,Private,4 Year,38530,51310,Ohio,51800,94000
725,Xavier University of Louisiana,Private,4 Year,24488,33535,Louisiana,46700,88200
726,Yale University,Private,4 Year,53430,69430,Connecticut,70300,138300


# Create database connection

In [7]:
rds_connection_string = "postgres:postgres@localhost:5432/salary_tuition"

engine = create_engine(f'postgresql://{rds_connection_string}')

In [8]:
engine.table_names()

/var/folders/8v/9xz9_bk116j2sr6hpcb2xbxm0000gn/T/ipykernel_43031/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['sal_tuition', 'salary', 'tuition']

# Load DataFrames into database

In [9]:
salary.to_sql(name='salary', con=engine, if_exists='replace', index=False)

In [10]:
pd.read_sql_query('SELECT * FROM salary', con = engine)

,name,state_name,early_career_pay,mid_career_pay
0,Auburn University,Alabama,54400,104500
1,University of Alabama in Huntsville,Alabama,57500,103900
2,The University of Alabama,Alabama,52300,97400
3,Tuskegee University,Alabama,54500,93500
4,Samford University,Alabama,48400,90500
...,...,...,...,...
930,Viterbo University,Wisconsin,46800,81900
931,Concordia University-Wisconsin,Wisconsin,46700,81600
932,University of Wisconsin-Parkside,Wisconsin,46000,81400
933,University of Wisconsin-River Falls,Wisconsin,47100,81300


In [11]:
tuition.to_sql(name='tuition', con=engine, if_exists='replace', index=False)

In [12]:
pd.read_sql_query('SELECT * FROM tuition', con = engine)

,name,state,type,degree_length,in_state_tuition,out_of_state_total
0,Adams State University,Colorado,Public,4 Year,9440,29238
1,Adventist University of Health Sciences,Florida,Private,4 Year,15150,19350
2,Agnes Scott College,Georgia,Private,4 Year,41160,53490
3,Alabama State University,Alabama,Public,4 Year,11068,24818
4,Alaska Pacific University,Alaska,Private,4 Year,20830,28130
...,...,...,...,...,...,...
723,Worcester Polytechnic Institute,Massachusetts,Private,4 Year,50530,65304
724,Xavier University,Ohio,Private,4 Year,38530,51310
725,Xavier University of Louisiana,Louisiana,Private,4 Year,24488,33535
726,Yale University,Connecticut,Private,4 Year,53430,69430


In [13]:
sal_tuition.to_sql(name='sal_tuition', con=engine, if_exists='replace', index=False)

In [14]:
pd.read_sql_query('SELECT * FROM sal_tuition', con = engine)

,name,state,type,degree_length,in_state_tuition,out_of_state_total,state_name,early_career_pay,mid_career_pay
0,Adams State University,Colorado,Public,4 Year,9440,29238,Colorado,44400,81400
1,Adventist University of Health Sciences,Florida,Private,4 Year,15150,19350,Florida,51600,89800
2,Agnes Scott College,Georgia,Private,4 Year,41160,53490,Georgia,46000,83600
3,Alabama State University,Alabama,Public,4 Year,11068,24818,Alabama,39800,71500
4,Alaska Pacific University,Alaska,Private,4 Year,20830,28130,Alaska,50300,90000
...,...,...,...,...,...,...,...,...,...
723,Worcester Polytechnic Institute,Massachusetts,Private,4 Year,50530,65304,Massachusetts,73600,135500
724,Xavier University,Ohio,Private,4 Year,38530,51310,Ohio,51800,94000
725,Xavier University of Louisiana,Louisiana,Private,4 Year,24488,33535,Louisiana,46700,88200
726,Yale University,Connecticut,Private,4 Year,53430,69430,Connecticut,70300,138300
